# 프로젝트 : 멋진 작사가 만들기

### Step 1. 데이터 다운로드
### Step 2. 데이터 읽어오기
### Step 3. 데이터 정제
### Step 4. 평가 데이터셋 분리
### Step 5. 인공지능 만들기

In [46]:
#데이터 읽어오기
import glob
import os, re
import numpy as np
import tensorflow as tf


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

#여러개의 txt파일을 모두 읽어서 raw_corpus에 담는다
for txt_file in txt_list:
    with open(txt_file, "r")as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:15])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream', 'As people always will', 'To be safe and warm', 'In that shining city on the hill Some wanna slam the door', 'Instead of opening the gate', "Aw, let's turn this thing around", 'Before it gets too late [Chorus]', "It's up to me and you", 'Love can conquer hate', 'I know this to be true', "That's what makes us great [Verse 2]"]


# 데이터 정제하기

In [47]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1", sentence)#2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+"," ", sentence) #4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample     sentence."))

<start> this is sample sentence . <end>


In [48]:
#정제된 문장을 모으는 곳
corpus = []

for sentence in raw_corpus:
    #우리가 원하지 않는 문장은 건너뛸 수 있게
    if len(sentence) == 0: continue  #길이가 0인 문장은 건너뜀
    if sentence[-1] == ":": continue 
    if sentence[0] == "[": continue # [verse]같은 건 건너뜀
    if sentence.count(" ") > 10 : continue    
        
    #정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
    
#정제된 결과를 10개만 확인
corpus[:10]
len(corpus)

150691

In [49]:
def tokenize(corpus):
   
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=23000, # 단어장의 크기설정 (12000이상 권장함)
        filters=' ', #이미 문장을 정제해서 filters필요x
        oov_token="<unk>" # 사전에 없는 단어는 <unk>로 
    )
    
    #corpus를 이용해 tokenizer 내부의 단어장을 완성하는 함수
    tokenizer.fit_on_texts(corpus)
    
    #준비한 tokenizer를 이용해서 corpus를 tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌(padding)
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용함
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   42   64 ...    0    0    0]
 [   2    9 2964 ...    0    0    0]
 [   2   42   64 ...    0    0    0]
 ...
 [   2  554   19 ...    0    0    0]
 [   2  121   36 ...    0    0    0]
 [   2    5   23 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc14ab80890>


In [50]:
#토큰화한 tensor
print(tensor[:5, :13])

[[   2   42   64   73  704    3    0    0    0    0    0    0    0]
 [   2    9 2964   10   27  262    3    0    0    0    0    0    0]
 [   2   42   64   10 2126  126   92    3    0    0    0    0    0]
 [   2   73  542   10 1068  542    8   42   25   76    9  344    3]
 [   2   82  173  171   84    3    0    0    0    0    0    0    0]]


In [51]:
#문장이 2로 시작하는데 알아보기 위해서 단어사전이 어떻게 구축되어있는지 아래코드로 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


#### 왜 시작이 2고 마지막이 3으로 끝나는지 알 수 있었음

## 데이터 셋 분리하기     
corpus 내의 첫 번째 문장에 대해 생성된 소스와 타겟 문장을 확인

In [52]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성하기
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성하기
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  42  64  73 704   3   0   0   0   0   0   0   0   0]
[ 42  64  73 704   3   0   0   0   0   0   0   0   0   0]


In [53]:
#train_test_split()함수를 사용해서 훈련데이터와 평가데이터 분리하기!
from sklearn.model_selection import train_test_split

#총 데이터의 20%를 평가 데이터셋으로 사용
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (120552, 14)
Target Train: (120552, 14)


## 데이터 셋 객체 생성

In [54]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만들기
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 만들기

* 모델의 Embedding Size와 Hidden Size를 조절해서 10Epoch안에     
val_loss값을 2.2 수준으로 줄일 수 있는 모델 설계하기

In [55]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [56]:
# 데이터 셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어보기
model(src_sample)

<tf.Tensor: shape=(256, 14, 23001), dtype=float32, numpy=
array([[[-1.43649726e-04,  1.76014903e-04,  2.42864189e-05, ...,
         -4.18973377e-06, -2.16621484e-05,  2.78816413e-04],
        [ 1.55447344e-06,  1.87925427e-04,  1.65282152e-04, ...,
          3.77308788e-05, -9.67640954e-05,  2.95939681e-04],
        [ 5.59317159e-05,  1.33576264e-04,  3.31338204e-04, ...,
         -1.47742467e-04, -2.78338703e-04,  2.11799226e-04],
        ...,
        [-3.86322761e-04, -1.00872094e-04,  2.04418422e-04, ...,
         -4.63991513e-04,  1.93131680e-04,  7.53880304e-04],
        [-6.51903916e-04, -5.36153166e-05,  1.37952316e-04, ...,
         -2.98109138e-04,  4.63124918e-04,  9.06863541e-04],
        [-6.22680876e-04,  1.14285380e-04,  1.95742759e-04, ...,
         -3.01188964e-04,  4.21511009e-04,  9.99791664e-04]],

       [[-1.43649726e-04,  1.76014903e-04,  2.42864189e-05, ...,
         -4.18973377e-06, -2.16621484e-05,  2.78816413e-04],
        [-2.14343265e-04,  5.61878915e-05, -1

In [57]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  5888256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  23576025  
Total params: 43,103,961
Trainable params: 43,103,961
Non-trainable params: 0
_________________________________________________________________


In [58]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
588/588 [==============================] - 289s 487ms/step - loss: 4.1214
Epoch 2/10
588/588 [==============================] - 287s 487ms/step - loss: 3.0614
Epoch 3/10
588/588 [==============================] - 287s 487ms/step - loss: 2.8863
Epoch 4/10
588/588 [==============================] - 286s 486ms/step - loss: 2.7556
Epoch 5/10
588/588 [==============================] - 286s 486ms/step - loss: 2.6464
Epoch 6/10
588/588 [==============================] - 285s 485ms/step - loss: 2.5510
Epoch 7/10
588/588 [==============================] - 285s 484ms/step - loss: 2.4647
Epoch 8/10
588/588 [==============================] - 285s 484ms/step - loss: 2.3744
Epoch 9/10
588/588 [==============================] - 284s 484ms/step - loss: 2.3019
Epoch 10/10
588/588 [==============================] - 284s 482ms/step - loss: 2.2271


In [59]:
#문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듦
    #    1. 입력받은 문장의 텐서를 입력하기
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냄
    #    3. 2에서 예측된 word index를 문장 뒤에 붙임
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성 마침
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [60]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i know", max_len=20)

'<start> i know you re smiling down on me <end> '

In [39]:
generate_text(model, tokenizer, init_sentence="<start> if we", max_len=20)

'<start> if we re not there <end> '